In [124]:
import re, os, sys
from functools import reduce
from itertools import combinations as comb, permutations as perm, combinations_with_replacement as combr
from operator import itemgetter
from pprint import pprint
from math import *
from collections import defaultdict
import networkx as nx
import numpy as np

def dprint(d):
    pprint(dict(d))
def lmap(*args, **kwargs):
    return list(map(*args, **kwargs))
def lmapi(*args, **kwargs):
    return list(map(int, *args, **kwargs))
def enum(*args, **kwargs):
    return enumerate(*args, **kwargs)

In [125]:
input_fname = "../inputs/day24-demo.in"

In [126]:
input_fname = "../inputs/day24.in"

In [127]:
def reread():
    lines = []
    with open(input_fname) as ifile:
        rows = []
        bliz = []
        for i, line in enum(ifile, start=-1):
            row = []
            for j, e in enum(line.strip(), start=-1):
                if e == "#": 
                    # continue
                    row.append(1)
                else:
                    row.append(0)
                    if e == ">":
                        bliz.append((1, j+i*1j))
                    elif e == "<":
                        bliz.append((-1, j+i*1j))
                    elif e == "^":
                        bliz.append((-1j, j+i*1j))
                    elif e == "v":
                        bliz.append((1j, j+i*1j))
            rows.append(row)
    return len(rows)-2, len(rows[0])-2, bliz

H, W, BLIZ = reread()
H, W, BLIZ[:5]

(25, 120, [(1, 0j), (1j, (1+0j)), (1, (2+0j)), ((-0-1j), (3+0j)), (1, (4+0j))])

In [128]:
def move(face, bpos):
    npos = bpos + face
    return (npos.real % W) + (npos.imag % H)*1j

In [129]:
def update_bliz(bliz):
    return [(face, move(face, bpos)) for face, bpos in bliz]

In [130]:
def isvalid(pos):
    return (0 <= pos.real < W) and (0 <= pos.imag < H)

## Part 1

In [131]:
from tqdm import trange

In [ ]:
period = 5*lcm(W, H)
d = nx.DiGraph()
bliz = update_bliz(BLIZ)
delta = (0, 1, -1, 1j, -1j)
for p in trange(period):
    newp = (p+2)%period
    nbliz = update_bliz(bliz)
    bposes = set(map(itemgetter(1), nbliz))
    for y in range(H):
        for x in range(W):
            pos = x+y*1j
            nposes = {pos + d for d in delta if isvalid(pos+d)}
            print("for", x, y, "at", p+1, "available:", nposes - bposes)
            for npos in nposes - bposes:
                newx, newy = int(npos.real), int(npos.imag)
                d.add_edge((p+1, x, y), (newp, newx, newy))
    bliz = nbliz

In [127]:
d.number_of_edges()

1317

In [98]:
start = (1, 0, 0)
end = (W-1, H-1)
for i in trange(2,600):
    tend = (i%period,) + end
    if tend in d and nx.has_path(d, start, tend):
        print("found path for i", i)
        break
else:
    print("didn't find any paths")

 42%|████▏     | 254/598 [00:57<01:17,  4.43it/s]

found path for i 256


## Part 2

## Faster version (WIP)
(with adjacency matrix based walk counting)

In [8]:
# coordinate tranformation helper funcs
period = lcm(W, H)
def c2i(t, x, y):
    return y*period*W + x*period + t
def i2c(i):
    t = i % period
    i = i // period
    x = i % W
    i = i // W
    y = i
    return t, x, y

In [30]:
# construct adjacency matrix and blizzard-free matrix
dim = period * W * H
ar = np.zeros((dim, dim), dtype=int)
bfm = np.ones((period, W, H), dtype=bool)
bliz = BLIZ.copy()
delta = (0, 1, -1, 1j, -1j)
for p in trange(period):
    newp = (p+1)%period
    bposes = set(map(itemgetter(1), bliz))
    nbliz = update_bliz(bliz)
    nbposes = set(map(itemgetter(1), nbliz))
    for y in range(H):
        for x in range(W):
            pos = x+y*1j
            if pos in bposes: 
                bfm[p,x,y] = False
                continue
            nposes = {pos + d for d in delta if isvalid(pos+d)}
            # print("for", x, y, "at", p+1, "available:", nposes - nbposes)
            for npos in nposes - nbposes:
                newx, newy = int(npos.real), int(npos.imag)
                ar[c2i(p, x, y), c2i(newp, newx, newy)] = 1
    bliz = nbliz

100%|██████████| 12/12 [00:00<00:00, 4960.74it/s]


In [31]:
i1 = c2i(0, W-1, H-1)
i2 = c2i(period-1, W-1, H-1)

In [32]:
from numpy.linalg import matrix_power

In [33]:
matrix_power(ar, 16)[c2i(1, 0, 0),i1:i2+1]

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0])

In [34]:
matrix_power(ar, 2)[c2i(2, 5, 3), c2i(4,5,1)]

1

In [38]:
list(enum(bfm[:,5,3]))

[(0, False),
 (1, False),
 (2, True),
 (3, True),
 (4, True),
 (5, True),
 (6, False),
 (7, False),
 (8, True),
 (9, True),
 (10, True),
 (11, True)]

In [73]:
ar[c2i(8,5,3),c2i(9,5,2)]

1

In [75]:
matrix_power(ar, 21)[c2i(9,5,3), c2i(5,0,0)]

0

In [102]:
def find_time(start, end, tstart):
    start = (tstart%period,) + start
    i1 = c2i(0, *end)
    i2 = c2i(period-1, *end)
    mar = ar.copy()
    for delta in range(1,1000):
        mar = mar @ ar
        if np.any(mar[c2i(*start), i1:i2+1]):
            fi = np.nonzero(mar[c2i(*start), i1:i2+1])[0][0]
            print(i2c(i1+fi))
            print(mar[c2i(*start), i1:i2+1])
            return delta
    return None

In [97]:
find_time((0, 0), (W-1, H-1), 1)

(5, 5, 3)
[0 0 0 0 0 2 0 0 0 0 0 0]


15

In [104]:
for i in range(period):
    print(i, find_time((W-1,H-1), (0, 0), i))

(4, 0, 0)
[ 0  0  0  0 10  0  0  0  0  0  0  0]
15
(4, 0, 0)
[ 0  0  0  0 10  0  0  0  0  0  0  0]
14


KeyboardInterrupt: 

## Better graph based version

In [132]:
period = lcm(W, H)
d = nx.DiGraph()
bliz = BLIZ.copy()
delta = (0, 1, -1, 1j, -1j)
for p in trange(period):
    nxtp = (p+1)%period
    blizlocs = set(map(itemgetter(1), bliz))
    nxtbliz = update_bliz(bliz)
    nxtblizlocs = set(map(itemgetter(1), nxtbliz))

    # src and dst
    d.add_edge((p, "src"), (nxtp, "src"))
    d.add_edge((p, "dst"), (nxtp, "dst"))
    nsrc, ndst = 0+0j, (W-1)+(H-1)*1j
    if nsrc not in blizlocs:
        d.add_edge((p, 0, 0), (nxtp, "src"))
    if nsrc not in nxtblizlocs:
        d.add_edge((p, "src"), (nxtp, 0, 0))
    if ndst not in blizlocs:
        d.add_edge((p, W-1, H-1), (nxtp, "dst"))
    if ndst not in nxtblizlocs:
        d.add_edge((p, "dst"), (nxtp, W-1, H-1))

    # between other locations
    for y in range(H):
        for x in range(W):
            pos = x+y*1j
            d.add_node(pos)
            if pos in blizlocs: continue
            nxtposes = {pos + d for d in delta if isvalid(pos+d)}
            # print("for", x, y, "at", p+1, "available:", nposes - nxtblizlocs)
            for nxtpos in nxtposes - nxtblizlocs:
                nxtx, nxty = int(nxtpos.real), int(nxtpos.imag)
                d.add_edge((p, x, y), (nxtp, nxtx, nxty))
    bliz = nxtbliz

100%|██████████| 600/600 [00:10<00:00, 54.70it/s]


In [133]:
nx.shortest_path_length(d, (0, "src"), (18, "dst"))

618

In [108]:
nx.shortest_path_length(d, (18, "dst"), (41%24, "src"))

23

In [135]:
def find_time(tstart, startpt, endpt):
    start = (tstart%period, startpt)
    for delta in trange(period):
        end = ((tstart+delta)%period, endpt)
        l = nx.shortest_path_length(d, start, end)
        if l == delta:
            return delta
    return -1

In [136]:
find_time(0, "src", "dst")

 43%|████▎     | 257/600 [02:38<03:31,  1.62it/s]


257

In [137]:
find_time(257, "dst", "src")

 51%|█████     | 307/600 [02:55<02:47,  1.75it/s]


307

In [138]:
find_time(257+307, "src", "dst")

 44%|████▍     | 264/600 [02:38<03:21,  1.67it/s]


264

In [139]:
257+307+264

828

In [122]:
def find_total_time():
    t1 = find_time(0, "src", "dst")
    t2 = find_time(t1, "dst", "src")
    t3 = find_time(t1+t2, "src", "dst")
    return t1 + t2 + t3

In [123]:
find_total_time()

54